# Perform single-cell level quality control

In [1]:
import pathlib
import sys

import pandas as pd
from cosmicqc import find_outliers

cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd
else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break
sys.path.append(str(root_dir / "utils"))
from arg_parsing_utils import parse_args
from notebook_init_utils import bandicoot_check, init_notebook

root_dir, in_notebook = init_notebook()

profile_base_dir = bandicoot_check(
    pathlib.Path("~/mnt/bandicoot").resolve(),
    # pathlib.Path("/home/jenna/mnt/bandicoot").resolve(), # for Jenna's machine
    root_dir,
)

## Load in each single-cell level profile per patient and process

1. Load in the single-cell data (add `patient_id` column).
2. Load in respective organoid qc data (only metadata and cqc columns) to already flag cells that come from a flagged organoid.
   - Also add a flag for if single-cells do not have an organoid segmentation (`parent_organoid` == -1).
   - Also add flag for if the `object_id` for a single-cell is NaN.
3. Concat single-cell data together.

In [2]:
# Path to patient folders
path_to_patients = pathlib.Path(f"{profile_base_dir}/data/")

dfs = []
for patient_folder in path_to_patients.iterdir():
    single_cell_file = (
        patient_folder / "image_based_profiles/1.combined_profiles" / "sc.parquet"
    )
    organoid_flags_file = (
        patient_folder
        / "image_based_profiles/1a.qc_profiles"
        / "organoid_flagged_outliers.parquet"
    )

    if single_cell_file.exists():
        sc_df = pd.read_parquet(single_cell_file)
        sc_df["patient_id"] = patient_folder.name

        # Default QC flags
        sc_df["cqc.organoid_flagged"] = False
        sc_df["cqc.nan_detected"] = sc_df["object_id"].isna()
        sc_df["cqc.missing_organoid"] = sc_df["parent_organoid"] == -1

        if organoid_flags_file.exists():
            organoid_flags_df = pd.read_parquet(organoid_flags_file)[
                ["object_id", "image_set"]
                + [
                    col
                    for col in pd.read_parquet(organoid_flags_file).columns
                    if col.startswith("cqc")
                ]
            ]

            # Get flagged (object_id, image_set) pairs
            flagged_pairs = set(
                organoid_flags_df.loc[
                    organoid_flags_df.filter(like="cqc").any(axis=1),
                    ["object_id", "image_set"],
                ].itertuples(index=False, name=None)
            )

            # Flag SC rows where both parent_organoid & image_set match a flagged organoid
            sc_df["cqc.organoid_flagged"] = sc_df.apply(
                lambda row: (row["parent_organoid"], row["image_set"]) in flagged_pairs,
                axis=1,
            )

        dfs.append(sc_df)

orig_single_cell_profiles_df = pd.concat(dfs, ignore_index=True)

print(orig_single_cell_profiles_df.shape)
orig_single_cell_profiles_df.head()

(11622, 1938)


/tmp/ipykernel_1386563/568601124.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  orig_single_cell_profiles_df = pd.concat(dfs, ignore_index=True)


,object_id,image_set,parent_organoid,Area.Size.Shape_Nuclei_VOLUME,Area.Size.Shape_Nuclei_CENTER.X,Area.Size.Shape_Nuclei_CENTER.Y,Area.Size.Shape_Nuclei_CENTER.Z,Area.Size.Shape_Nuclei_BBOX.VOLUME,Area.Size.Shape_Nuclei_MIN.X,Area.Size.Shape_Nuclei_MAX.X,...,Intensity_Nuclei_DNA_MIN.INTENSITY_y,Intensity_Nuclei_DNA_MIN.INTENSITY.EDGE_y,Intensity_Nuclei_DNA_STD.INTENSITY_y,Intensity_Nuclei_DNA_STD.INTENSITY.EDGE_y,Intensity_Nuclei_DNA_UPPER.QUARTILE.INTENSITY_y,Intensity_Nuclei_DNA_VOLUME_y,patient_id,cqc.organoid_flagged,cqc.nan_detected,cqc.missing_organoid
0,<NA>,C11-4,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NF0040_T1,False,True,<NA>
1,42,D11-5,10,35476.0,811.874390,478.418701,5.210170,67551.0,767.0,856.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NF0040_T1,False,False,False
2,85,D11-5,-1,36789.0,87.725243,495.730042,6.407622,83886.0,43.0,136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NF0040_T1,False,False,True
3,127,D11-5,-1,3606.0,716.765381,1025.581787,1.500000,4956.0,686.0,745.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NF0040_T1,False,False,True
4,212,D11-5,10,35259.0,785.665894,521.811523,6.219008,97416.0,751.0,833.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NF0040_T1,False,False,False


## Detect outlier single-cells using the non-flagged data

We will attempt to detect instances of poor quality segmentations using the nuclei compartment as the base. The conditions we are using are as follows:

1. Abnormally small or large nuclei using `Volume`
2. Abnormally high `mass displacement` in the nuclei for instances of mis-segmentation of background/no longer in-focus

In [3]:
# Set the metadata columns to be used in the QC process
metadata_columns = [
    "patient_id",
    "image_set",
    "object_id",
    "parent_organoid",
    "Area.Size.Shape_Nuclei_CENTER.X",
    "Area.Size.Shape_Nuclei_CENTER.Y",
    "cqc.nan_detected",
    "cqc.organoid_flagged",
    "cqc.missing_organoid",
]

In [4]:
# Process each plate (patient_id) independently in the combined dataframe
for plate_name, plate_df in orig_single_cell_profiles_df.groupby("patient_id"):
    print(f"Processing plate: {plate_name}")

    # Make a contiguous copy to prevent DataFrame fragmentation
    plate_df = plate_df.copy()

    # Only process the rows that are not flagged
    filtered_plate_df = plate_df[
        ~(
            plate_df["cqc.nan_detected"]
            | plate_df["cqc.organoid_flagged"]
            | plate_df["cqc.missing_organoid"]
        )
    ]

    # --- Find size based nuclei outliers ---
    print("Finding small nuclei outliers...")
    small_nuclei_outliers = find_outliers(
        df=filtered_plate_df,
        metadata_columns=metadata_columns,
        feature_thresholds={
            "Area.Size.Shape_Nuclei_VOLUME": -1,  # Detect very small nuclei
        },
    )

    # Ensure the column exists before assignment
    plate_df["cqc.small_nuclei_outlier"] = False
    plate_df.loc[small_nuclei_outliers.index, "cqc.small_nuclei_outlier"] = True

    print("Finding large nuclei outliers...")
    large_nuclei_outliers = find_outliers(
        df=filtered_plate_df,
        metadata_columns=metadata_columns,
        feature_thresholds={
            "Area.Size.Shape_Nuclei_VOLUME": 2,  # Detect very large nuclei
        },
    )

    # Ensure the column exists before assignment
    plate_df["cqc.large_nuclei_outlier"] = False
    plate_df.loc[large_nuclei_outliers.index, "cqc.large_nuclei_outlier"] = True

    # --- Find mass displacement based nuclei outliers ---
    print("Finding high mass displacement outliers...")
    high_mass_displacement_outliers = find_outliers(
        df=filtered_plate_df,
        metadata_columns=metadata_columns,
        feature_thresholds={
            "Intensity_Nuclei_DNA_MASS.DISPLACEMENT": 2,  # Detect high mass displacement
        },
    )

    # Ensure the column exists before assignment
    plate_df["cqc.mass_displacement_outlier"] = False
    plate_df.loc[
        high_mass_displacement_outliers.index, "cqc.mass_displacement_outlier"
    ] = True

    # Print number of outliers (only in filtered rows)
    small_count = filtered_plate_df.index.intersection(
        small_nuclei_outliers.index
    ).shape[0]
    large_count = filtered_plate_df.index.intersection(
        large_nuclei_outliers.index
    ).shape[0]
    high_mass_count = filtered_plate_df.index.intersection(
        high_mass_displacement_outliers.index
    ).shape[0]

    print(f"Small nuclei outliers found: {small_count}")
    print(f"Large nuclei outliers found: {large_count}")
    print(f"High mass displacement outliers found: {high_mass_count}")

    # Save updated plate_df with flag columns included
    output_folder = (
        path_to_patients / plate_name / "image_based_profiles/1a.qc_profiles"
    )
    output_folder.mkdir(parents=True, exist_ok=True)
    output_file = output_folder / "sc_flagged_outliers.parquet"
    plate_df.to_parquet(output_file, index=False)
    print(f"Saved single-cell profiles with outlier flags to {output_file}\n")

Processing plate: NF0014_T1
Finding small nuclei outliers...
Number of outliers: 175 (14.57%)
Outliers Range:
Area.Size.Shape_Nuclei_VOLUME Min: 350.0
Area.Size.Shape_Nuclei_VOLUME Max: 19155.0
Finding large nuclei outliers...
Number of outliers: 67 (5.58%)
Outliers Range:
Area.Size.Shape_Nuclei_VOLUME Min: 163455.0
Area.Size.Shape_Nuclei_VOLUME Max: 310499.0
Finding high mass displacement outliers...
Number of outliers: 0 (0.00%)
Outliers Range:
Intensity_Nuclei_DNA_MASS.DISPLACEMENT Min: nan
Intensity_Nuclei_DNA_MASS.DISPLACEMENT Max: nan
Small nuclei outliers found: 175
Large nuclei outliers found: 67
High mass displacement outliers found: 0
Saved single-cell profiles with outlier flags to ~/mnt/bandicoot/NF1_organoid_data/data/NF0014_T1/image_based_profiles/1a.qc_profiles/sc_flagged_outliers.parquet

Processing plate: NF0016_T1
Finding small nuclei outliers...
Number of outliers: 43 (16.10%)
Outliers Range:
Area.Size.Shape_Nuclei_VOLUME Min: 355.0
Area.Size.Shape_Nuclei_VOLUME Max:

In [5]:
# Print example output of the flagged single-cell profiles
print(f"Example flagged single-cell profiles: {plate_name}")
print(plate_df.shape)
plate_df.head()

Example flagged single-cell profiles: SARCO361_T1
(1582, 1941)


,object_id,image_set,parent_organoid,Area.Size.Shape_Nuclei_VOLUME,Area.Size.Shape_Nuclei_CENTER.X,Area.Size.Shape_Nuclei_CENTER.Y,Area.Size.Shape_Nuclei_CENTER.Z,Area.Size.Shape_Nuclei_BBOX.VOLUME,Area.Size.Shape_Nuclei_MIN.X,Area.Size.Shape_Nuclei_MAX.X,...,Intensity_Nuclei_DNA_STD.INTENSITY.EDGE_y,Intensity_Nuclei_DNA_UPPER.QUARTILE.INTENSITY_y,Intensity_Nuclei_DNA_VOLUME_y,patient_id,cqc.organoid_flagged,cqc.nan_detected,cqc.missing_organoid,cqc.small_nuclei_outlier,cqc.large_nuclei_outlier,cqc.mass_displacement_outlier
4299,<NA>,C11-4,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,SARCO361_T1,True,True,<NA>,False,False,False
4300,42,D11-5,-1,6588.0,601.517761,816.406677,1.333333,11564.0,572.0,631.0,...,NaN,NaN,NaN,SARCO361_T1,False,False,True,False,False,False
4301,85,D11-5,1,40142.0,1082.144287,655.350159,5.550296,1264788.0,572.0,1160.0,...,NaN,NaN,NaN,SARCO361_T1,True,False,False,False,False,False
4302,127,D11-5,1,39224.0,539.322693,880.315125,6.072481,68000.0,496.0,581.0,...,NaN,NaN,NaN,SARCO361_T1,True,False,False,False,False,False
4303,255,D11-5,-1,11643.0,520.695801,491.240387,8.148072,19610.0,495.0,548.0,...,NaN,NaN,NaN,SARCO361_T1,False,False,True,False,False,False
